# baseline
https://mp.weixin.qq.com/s/_40sp7g6dHptwWyHvSygMg

重新添加了一个voc的最短电话时间间隔的强特征，一下子提升了5个百分点

https://mp.weixin.qq.com/s/zmLlHX9z5_Q-juJSp2bNyw

https://github.com/LogicJake/competition_baselines/blob/master/competitions/dc_fraud_phonecall/EDA_and_Baseline.ipynb

voc、sms 除了统计特征，还可以做时间间隔特征等；

app 可以重点关注一些应用的流量，甚至可以像今年腾讯赛一样做 w2v；

city、county 可以做其他的编码方式；

其他常规的融合和 stacking。

# baseline lgb+5fold
https://github.com/biaobiao2/DC_phone/blob/master/code/baseline.py

思路比较常规，先做统计特征，然后5折

还是有一些可以借鉴的地方

随便提交了一下，rank120。。。

In [1]:
'''
Created on 2020年6月5日
@author: LSH
'''
import os
import gc
import time
import psutil
import datetime
import numpy as np
import pandas as pd
import lightgbm as lgb
import pyarrow.parquet as pq
from gensim.models import Word2Vec
from collections import defaultdict
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from scipy.stats import entropy, pearsonr, stats
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from code.config import * 
pd.set_option('display.unicode.ambiguous_as_wide', True)
pd.set_option('display.unicode.east_asian_width', True)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option("display.max_colwidth",100)
pd.set_option('display.width',1000)

In [2]:
args.DATA_DIR

'../../data/contest/20sichuan_fraudtel_detection/'

In [ ]:
path = args.DATA_DIR
user_data = args.DATA_DIR+'data_gen'

# fe

## get_app_feats

In [ ]:
def get_app_feats(df):
#     print(df.head())
#     print(df["busi_name"].value_counts())
    phones_app = df[["phone_no_m"]].copy()
    phones_app = phones_app.drop_duplicates(subset=['phone_no_m'], keep='last')
    # 每个user使用app的nunique
    tmp = df.groupby("phone_no_m")["busi_name"].agg(busi_count="nunique")
    phones_app = phones_app.merge(tmp, on="phone_no_m", how="left")
    """使用的流量统计
    """
    tmp = df.groupby("phone_no_m")["flow"].agg(flow_mean="mean", 
                                               flow_median = "median", 
                                               flow_min  = "min", 
                                               flow_max = "max", 
                                               flow_var = "var",
                                               flow_sum = "sum")
    phones_app = phones_app.merge(tmp, on="phone_no_m", how="left")
    # 所在月份数
    tmp = df.groupby("phone_no_m")["month_id"].agg(month_ids ="nunique")
    phones_app = phones_app.merge(tmp, on="phone_no_m", how="left")
    #月流量使用统计
    phones_app["flow_month"] = phones_app["flow_sum"] / phones_app["month_ids"]

    return phones_app

## get_voc_feat

In [ ]:
def get_voc_feat(df):
    df["start_datetime"] = pd.to_datetime(df['start_datetime'] )
    df["hour"] = df['start_datetime'].dt.hour
    df["day"] = df['start_datetime'].dt.day
    print(df.head())
    phone_no_m = df[["phone_no_m"]].copy()
    phone_no_m = phone_no_m.drop_duplicates(subset=['phone_no_m'], keep='last')
    #对话人数和对话次数
    tmp = df.groupby("phone_no_m")["opposite_no_m"].agg(opposite_count="count", opposite_unique="nunique")
    phone_no_m = phone_no_m.merge(tmp, on="phone_no_m", how="left")
    """主叫通话
    """
    df_call = df[df["calltype_id"]==1].copy()
    tmp = df_call.groupby("phone_no_m")["imei_m"].agg(voccalltype1="count", imeis="nunique")
    phone_no_m = phone_no_m.merge(tmp, on="phone_no_m", how="left")
    phone_no_m["voc_calltype1"] = phone_no_m["voccalltype1"] / phone_no_m["opposite_count"] 
    tmp = df_call.groupby("phone_no_m")["city_name"].agg(city_name_call="nunique")
    phone_no_m = phone_no_m.merge(tmp, on="phone_no_m", how="left")
    tmp = df_call.groupby("phone_no_m")["county_name"].agg(county_name_call="nunique")
    phone_no_m = phone_no_m.merge(tmp, on="phone_no_m", how="left")
    
    """和固定通话者的对话统计
    """
    tmp = df.groupby(["phone_no_m","opposite_no_m"])["call_dur"].agg(count="count", sum="sum")
    phone2opposite = tmp.groupby("phone_no_m")["count"].agg(phone2opposite_mean="mean", phone2opposite_median="median", phone2opposite_max="max")
    phone_no_m = phone_no_m.merge(phone2opposite, on="phone_no_m", how="left")
    phone2opposite = tmp.groupby("phone_no_m")["sum"].agg(phone2oppo_sum_mean="mean", phone2oppo_sum_median="median", phone2oppo_sum_max="max")
    phone_no_m = phone_no_m.merge(phone2opposite, on="phone_no_m", how="left")
    
    """通话时间长短统计
    """
    tmp = df.groupby("phone_no_m")["call_dur"].agg(call_dur_mean="mean", call_dur_median="median", call_dur_max="max", call_dur_min="min")
    phone_no_m = phone_no_m.merge(tmp, on="phone_no_m", how="left")
    
    tmp = df.groupby("phone_no_m")["city_name"].agg(city_name_nunique="nunique")
    phone_no_m = phone_no_m.merge(tmp, on="phone_no_m", how="left")
    tmp = df.groupby("phone_no_m")["county_name"].agg(county_name_nunique="nunique")
    phone_no_m = phone_no_m.merge(tmp, on="phone_no_m", how="left")
    tmp = df.groupby("phone_no_m")["calltype_id"].agg(calltype_id_unique="nunique")
    phone_no_m = phone_no_m.merge(tmp, on="phone_no_m", how="left")
    
    """通话时间点偏好
    """
    tmp = df.groupby("phone_no_m")["hour"].agg(voc_hour_mode = lambda x:stats.mode(x)[0][0], 
                                               voc_hour_mode_count = lambda x:stats.mode(x)[1][0], 
                                               voc_hour_nunique="nunique")
    phone_no_m = phone_no_m.merge(tmp, on="phone_no_m", how="left")
    
    tmp = df.groupby("phone_no_m")["day"].agg(voc_day_mode = lambda x:stats.mode(x)[0][0], 
                                               voc_day_mode_count = lambda x:stats.mode(x)[1][0], 
                                               voc_day_nunique="nunique")
    phone_no_m = phone_no_m.merge(tmp, on="phone_no_m", how="left")
    return phone_no_m

## get_sms_feats

In [ ]:
def get_sms_feats(df):
    print(df.head())
    df['request_datetime'] = pd.to_datetime(df['request_datetime'] )
    df["hour"] = df['request_datetime'].dt.hour
    df["day"] = df['request_datetime'].dt.day
#     df["month"] = df['request_datetime'].dt.month

    phone_no_m = df[["phone_no_m"]].copy()
    phone_no_m = phone_no_m.drop_duplicates(subset=['phone_no_m'], keep='last')
    #对话人数和对话次数
    tmp = df.groupby("phone_no_m")["opposite_no_m"].agg(sms_count="count", sms_nunique="nunique")
    tmp["sms_rate"] = tmp["sms_count"]/tmp["sms_nunique"]
    phone_no_m = phone_no_m.merge(tmp, on="phone_no_m", how="left")
    """短信下行比例
    """
    calltype2 = df[df["calltype_id"]==2].copy()
    calltype2 = calltype2.groupby("phone_no_m")["calltype_id"].agg(calltype_2="count")
    phone_no_m = phone_no_m.merge(calltype2, on="phone_no_m", how="left")
    phone_no_m["calltype_rate"] = phone_no_m["calltype_2"] / phone_no_m["sms_count"]
    """短信时间
    """
    tmp = df.groupby("phone_no_m")["hour"].agg(hour_mode = lambda x:stats.mode(x)[0][0], 
                                               hour_mode_count = lambda x:stats.mode(x)[1][0], 
                                               hour_nunique="nunique")
    phone_no_m = phone_no_m.merge(tmp, on="phone_no_m", how="left")
    
    tmp = df.groupby("phone_no_m")["day"].agg(day_mode = lambda x:stats.mode(x)[0][0], 
                                               day_mode_count = lambda x:stats.mode(x)[1][0], 
                                               day_nunique="nunique")
    phone_no_m = phone_no_m.merge(tmp, on="phone_no_m", how="left")
    
    return phone_no_m

## feats

In [ ]:
def feats():
    test_voc=pd.read_csv(path+'test/test_voc.csv',)
    test_voc_feat = get_voc_feat(test_voc)
    test_voc_feat.to_csv(user_data + "test_voc_feat.csv", index=False)

    test_app=pd.read_csv(path+'test/test_app.csv',)
    test_app_feat = get_app_feats(test_app)
    test_app_feat.to_csv(user_data + "test_app_feat.csv", index=False)
    
    test_sms=pd.read_csv(path+'test/test_sms.csv',)
    test_sms_feat = get_sms_feats(test_sms)
    test_sms_feat.to_csv(user_data + "test_sms_feat.csv", index=False)
     
    train_voc=pd.read_csv(path+'train/train_voc.csv',)
    train_voc_feat = get_voc_feat(train_voc)
    train_voc_feat.to_csv(user_data + "train_voc_feat.csv", index=False)

    train_app=pd.read_csv(path+'train/train_app.csv',)
    train_app_feat = get_app_feats(train_app)
    train_app_feat.to_csv(user_data + "train_app_feat.csv", index=False)

    train_sms=pd.read_csv(path+'train/train_sms.csv',)
    train_sms_feat = get_sms_feats(train_sms)
    train_sms_feat.to_csv(user_data + "train_sms_feat.csv", index=False)

In [10]:
%%time
feats()

                                                                                            phone_no_m                                                                                        opposite_no_m  calltype_id      start_datetime  call_dur city_name county_name                                                                                               imei_m  hour  day
0  b3dce36871f3e88164b18d4953b114163f008cb51c28fe932ae3c734f3b1d1e2853d63fb3fb52a09fd9d0997b64fe579...  f87f526ee776ac8b6b28392620fbb6049af9eaadab7de2ee9d7c5a31d7404566b8a3c25f404f793e513448a8c5dad194...            1 2020-04-13 21:04:10      1909       NaN         NaN  685a0bfcd91b4ecec2ff35d656a35a4e4f9e1c7a92b15f5c3c0675fcf8b6c9adc35731ab15e91401f2367f1d9d2910f9...    21   13
1  b3dce36871f3e88164b18d4953b114163f008cb51c28fe932ae3c734f3b1d1e2853d63fb3fb52a09fd9d0997b64fe579...  f87f526ee776ac8b6b28392620fbb6049af9eaadab7de2ee9d7c5a31d7404566b8a3c25f404f793e513448a8c5dad194...            1 2020-04-13 18:32:50  

                                                                                            phone_no_m                                                                                        opposite_no_m  calltype_id     request_datetime
0  ce0dfda7499c30385ad400360a96476b2bbb16484e308f3f74abccf67857a7c293fe240b5b959b8b81c3bcdc4b6996d9...  18f2150b69516bbf52f6593aed24b01333936f64cd3f8f5a2ca2bb64c5dea32ed41f3f7bea71fe7c3cd1d9f6e9da4e82...            2  2020-04-01 20:13:01
1  ce0dfda7499c30385ad400360a96476b2bbb16484e308f3f74abccf67857a7c293fe240b5b959b8b81c3bcdc4b6996d9...  df22edbc0e3dd6bc4f2f453e687b743e8442a54834b64f06ee093a91c4a41385d46579107a443eee3ec0ce2339bcea41...            2  2020-04-01 13:51:50
2  ce0dfda7499c30385ad400360a96476b2bbb16484e308f3f74abccf67857a7c293fe240b5b959b8b81c3bcdc4b6996d9...  df22edbc0e3dd6bc4f2f453e687b743e8442a54834b64f06ee093a91c4a41385d46579107a443eee3ec0ce2339bcea41...            2  2020-04-01 13:51:50
3  ce0dfda7499c30385ad400360a96476b2bbb16484e308

/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


                                                                                            phone_no_m                                                                                        opposite_no_m  calltype_id      start_datetime  call_dur city_name county_name                                                                                               imei_m  hour  day
0  3b6bef554a4754fa9977b50d99e6777c9fb26e585961a453bab3a2eba9f25eb510376bb89a345fb1a073e3a99da296d0...  3572f19c52faeea5111ddd79dfc3643171d00e6d8b3fd9d6d04701a1873702a50004019f70d8a4c766d24e7768d356f6...            2 2019-10-17 17:16:25        13       NaN         NaN  d2940d7cde5092d809af2e4bb8f6936a405382bd541542c5ccbb9d577d7d340ea032cca1174d33cca05393fbfd45fc07...    17   17
1  3b6bef554a4754fa9977b50d99e6777c9fb26e585961a453bab3a2eba9f25eb510376bb89a345fb1a073e3a99da296d0...  3572f19c52faeea5111ddd79dfc3643171d00e6d8b3fd9d6d04701a1873702a50004019f70d8a4c766d24e7768d356f6...            2 2019-10-17 17:36:20  

                                                                                            phone_no_m                                                                                        opposite_no_m  calltype_id     request_datetime
0  0251387744988114430181c3e680a3733001a26fe686bdc1e64dfeaecd5d9e6f137df7f2a8d0a5871bcf6f371cab5bd3...  df22edbc0e3dd6bc4f2f453e687b743e8442a54834b64f06ee093a91c4a41385d46579107a443eee3ec0ce2339bcea41...            2  2019-08-01 12:13:08
1  0251387744988114430181c3e680a3733001a26fe686bdc1e64dfeaecd5d9e6f137df7f2a8d0a5871bcf6f371cab5bd3...  df22edbc0e3dd6bc4f2f453e687b743e8442a54834b64f06ee093a91c4a41385d46579107a443eee3ec0ce2339bcea41...            2  2019-08-01 12:13:08
2  0251387744988114430181c3e680a3733001a26fe686bdc1e64dfeaecd5d9e6f137df7f2a8d0a5871bcf6f371cab5bd3...  df22edbc0e3dd6bc4f2f453e687b743e8442a54834b64f06ee093a91c4a41385d46579107a443eee3ec0ce2339bcea41...            2  2019-08-01 12:13:08
3  296cfae1d838070c4dd05a125a85c3d29bbb95f713c2e

In [41]:
train_app_feat = pd.read_csv(user_data + "train_app_feat.csv")
train_voc_feat = pd.read_csv(user_data + "train_voc_feat.csv")
train_sms_feat = pd.read_csv(user_data + "train_sms_feat.csv")
train_user = pd.read_csv(path+'train/train_user.csv')
drop_r = ["arpu_201908", "arpu_201909", "arpu_201910",
          "arpu_201911", "arpu_201912", "arpu_202001", "arpu_202002"]
train_user.drop(drop_r, axis=1, inplace=True)
train_user.rename(columns={"arpu_202003": "arpu_202004"}, inplace=True)
train_user["city_name"] = LabelEncoder().fit_transform(
    train_user["city_name"].astype(np.str))
train_user["county_name"] = LabelEncoder().fit_transform(
    train_user["county_name"].astype(np.str))

In [42]:
train_app_feat.head(2)

,phone_no_m,busi_count,flow_mean,flow_median,flow_min,flow_max,flow_var,flow_sum,month_ids,flow_month
0,416cec0f25b93f08bfd9cff44382c5da3a6346beb16a2c97d60916a358b15084b51c244fbf7ed461752faaac57a95ef7...,0,0.0,0.0,0.0,0.0,NaN,0.0,0,NaN
1,26fcb7c6f4125ee5445756d4ff5346c29f2aff7d0f2e3171cb74ee47fc5d3087a76371120882b5a7698e07467ffad7ed...,0,0.0,0.0,0.0,0.0,NaN,0.0,0,NaN


In [43]:
train_voc_feat.head(2)

,phone_no_m,opposite_count,opposite_unique,voccalltype1,imeis,voc_calltype1,city_name_call,county_name_call,phone2opposite_mean,phone2opposite_median,phone2opposite_max,phone2oppo_sum_mean,phone2oppo_sum_median,phone2oppo_sum_max,call_dur_mean,call_dur_median,call_dur_max,call_dur_min,city_name_nunique,county_name_nunique,calltype_id_unique,voc_hour_mode,voc_hour_mode_count,voc_hour_nunique,voc_day_mode,voc_day_mode_count,voc_day_nunique
0,b8758a7e6a4834bd69444ef669a5bb21a8b352d0a4759021bf9291a50cf981dc247d2ba5235987963804958952df69f0...,65,59,65.0,1.0,1.0,0.0,0.0,1.101695,1.0,3,143.389831,43.0,2372,130.153846,42.0,1800,3,0,0,1,16,24,5,25,65,1
1,b6cd606931ba69138706e76f1bb691c13ae2916ddb2a2df84825c75ff6d169be8290f45612b387365fb7096aae4b11fc...,1,1,1.0,1.0,1.0,0.0,0.0,1.000000,1.0,1,14.000000,14.0,14,14.000000,14.0,14,14,0,0,1,16,1,1,3,1,1


In [44]:
train_sms_feat.head(2)

,phone_no_m,sms_count,sms_nunique,sms_rate,calltype_2,calltype_rate,hour_mode,hour_mode_count,hour_nunique,day_mode,day_mode_count,day_nunique
0,30fff49d3fbf020411af17c84ee2912a0bfd9e3036e50a8297255571e02513e5f8560ba46188e9f911b1ec813d183da2...,2,2,1.0,1.0,0.5,10,2,1,19,2,1
1,80d5353ac17cab91ddb0c1f8d3aec1e84420699230a800891a82b5922c126650327d351eedf7f0c0c330f5d93dc3df1b...,14,5,2.8,14.0,1.0,13,10,2,9,14,1


In [45]:
train_user.head(2)

,phone_no_m,city_name,county_name,idcard_cnt,arpu_202004,label
0,672ddbf02a5544d32e4ecc9433b1981bffe23bf912273a3a835f6cccb78b8ed7554e9ab0fbcd33d19eb6063ce00542dd...,16,94,1,45.0,0
1,5e1272273e041e82cb275ae877710be98cdaf5b0a8f34de8d361f71d5268fa0851edffd3950e170df1e3846fcf90cc7c...,10,79,1,60.0,0


In [39]:
test_app_feat = pd.read_csv(user_data+'test_app_feat.csv')
test_voc_feat = pd.read_csv(user_data+'test_voc_feat.csv')
test_sms_feat = pd.read_csv(user_data + "test_sms_feat.csv")
test_user = pd.read_csv(path+'test/test_user.csv')
test_user["city_name"] = LabelEncoder().fit_transform(
    test_user["city_name"].astype(np.str))
test_user["county_name"] = LabelEncoder().fit_transform(
    test_user["county_name"].astype(np.str))

In [40]:
test_app_feat.head(2)

,phone_no_m,busi_count,flow_mean,flow_median,flow_min,flow_max,flow_var,flow_sum,month_ids,flow_month
0,2abe25ed833883fcb81fb9f987596af936a7ef922c8d1de33415928ae5c5a5fce06e92e805cb08676a3dd6554a01c1c7...,167,37.123944,0.274308,0.000656,1662.155730,29294.221717,6385.318305,1,6385.318305
1,0fbe23bde8d6f1d6bb1895fdf77cbb45b66a83edb81179cf701ef920a57ef9b1da25ff5e83ec4c4204c083f94652811a...,1,0.144213,0.144213,0.144213,0.144213,NaN,0.144213,1,0.144213


In [ ]:

test_user = test_user.merge(test_app_feat, on="phone_no_m", how="left")
test_user = test_user.merge(test_voc_feat, on="phone_no_m", how="left")
test_user = test_user.merge(test_sms_feat, on="phone_no_m", how="left")


train_user = train_user.merge(train_app_feat, on="phone_no_m", how="left")
train_user = train_user.merge(train_voc_feat, on="phone_no_m", how="left")
train_user = train_user.merge(train_sms_feat, on="phone_no_m", how="left")

# train_model
## f1阈值搜索score_vail

In [ ]:
def score_vail(vaild_preds, real):
    """f1阈值搜索
    """
#     import matplotlib.pylab as plt
#     plt.figure(figsize=(16,5*10))
    best = 0
    bp = 0
    score = []
    for i in range(600):
        p = 32+i*0.08 # p从32遍历搜索到80 
        threshold_test = round(np.percentile(vaild_preds, p), 4)
        pred_int = vaild_preds>threshold_test
        ff = f1_score(pred_int,real)
        score.append(ff)
        
        if ff>=best:
            best = ff
            bp = p
#     plt.plot(range(len(score)), score)
#     plt.show()
    return bp, best

## train_model

In [48]:
train_user["city_name"].astype(np.str)

0       16
1       10
2       11
3       11
4       10
5       19
6       11
7        1
8       10
9       11
10      19
11      11
12      11
13      11
14      15
15       6
16      11
17      19
18      11
19       4
20      11
21      11
22      13
23       1
24      22
25      11
26       1
27       6
28      11
29      11
30      11
31      20
32       6
33      20
34      11
35      11
36      11
37      11
38      15
39      11
40      11
41       9
42      11
43      16
44       9
45      11
46      16
47      11
48      13
49      11
50      22
51      11
52       5
53      12
54       4
55      16
56      11
57       5
58      11
59      11
60      19
61      10
62      22
63       9
64       3
65       4
66      20
67      11
68      11
69       0
70      10
71      10
72       5
73      19
74      16
75       0
76       5
77       7
78      17
79      20
80       5
81       5
82      21
83      11
84       5
85      11
86      11
87      13
88       9
89       4
90      19

In [72]:
def train_model():
    train_app_feat = pd.read_csv(user_data + "train_app_feat.csv")
    train_voc_feat = pd.read_csv(user_data + "train_voc_feat.csv")
    train_sms_feat=pd.read_csv(user_data + "train_sms_feat.csv")
    train_user=pd.read_csv(path+'train/train_user.csv')
    drop_r = ["arpu_201908","arpu_201909","arpu_201910","arpu_201911","arpu_201912","arpu_202001","arpu_202002"]
    train_user.drop(drop_r, axis=1,inplace=True)
    train_user.rename(columns={"arpu_202003":"arpu_202004"},inplace=True)
    train_user = train_user.merge(train_app_feat, on="phone_no_m", how="left")
    train_user = train_user.merge(train_voc_feat, on="phone_no_m", how="left")
    train_user = train_user.merge(train_sms_feat, on="phone_no_m", how="left")
    
    test_app_feat=pd.read_csv(user_data+'test_app_feat.csv')
    test_voc_feat=pd.read_csv(user_data+'test_voc_feat.csv')
    test_sms_feat=pd.read_csv(user_data + "test_sms_feat.csv")
    test_user=pd.read_csv(path+'test/test_user.csv')
    test_user = test_user.merge(test_app_feat, on="phone_no_m", how="left")
    test_user = test_user.merge(test_voc_feat, on="phone_no_m", how="left")
    test_user = test_user.merge(test_sms_feat, on="phone_no_m", how="left")
    
    lbe_city = LabelEncoder()
    lbe_county = LabelEncoder()
    lbe_city.fit(pd.Series(list(set(train_user["city_name"]).union(set(test_user["city_name"])))).astype(np.str))
    lbe_county.fit(pd.Series(list(set(train_user["county_name"]).union(set(test_user["county_name"])))).astype(np.str))
    train_user["city_name"] = lbe_city.transform(train_user["city_name"].astype(np.str))
    train_user["county_name"] = lbe_county.transform(train_user["county_name"].astype(np.str))

    test_user["city_name"] = lbe_city.transform(test_user["city_name"].astype(np.str))
    test_user["county_name"] = lbe_county.transform(test_user["county_name"].astype(np.str))
    
    sub = test_user[["phone_no_m"]].copy()
    
    params = {
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'max_bin': 50,
    'max_depth': 16,
    "learning_rate":0.1,
    "colsample_bytree":0.8,#每次迭代中随机选择特征的比例
    'min_child_samples': 25,
    'verbosity': -1,    #信息输出设置成1则没有信息输出
    'seed':1024,
    }
    train_label = train_user[["label"]].copy()
    print(np.sum(train_label), train_label.shape, train_label.head())
    
    test_user.drop(["phone_no_m"], axis=1,inplace=True)
    train_user.drop(["phone_no_m", "label"], axis=1,inplace=True)
    kf = StratifiedKFold(n_splits = 5, shuffle=True, random_state=2020)
    test_p = []
    threshold = 0
    scores = 0
    for i, (train_index, vaild_index) in enumerate(kf.split(train_user, train_label["label"])):
#         print('第{}次训练...'.format(i+1))
        train_x = train_user.iloc[train_index]
        train_y = train_label.iloc[train_index]
        valid_x = train_user.iloc[vaild_index]
        valid_y = train_label.iloc[vaild_index]
        
        cat = ["city_name", "county_name"]
        lgb_train = lgb.Dataset(train_x, train_y, categorical_feature=cat, silent=True)
        lgb_eval = lgb.Dataset(valid_x, valid_y, reference=lgb_train, categorical_feature=cat, silent=True)
        
        gbm = lgb.train(params, lgb_train, num_boost_round=1000, valid_sets=[lgb_train,lgb_eval], verbose_eval=100, early_stopping_rounds=200)
    
        vaild_preds = gbm.predict(valid_x, num_iteration=gbm.best_iteration)
        bp, bestf1= score_vail(vaild_preds, valid_y)
        print("score: ", bestf1 , bp)
        scores += bestf1
        threshold += bp
#         sauc = myFevalAuc(vaild_preds, valid_y)
        importance = gbm.feature_importance()
        feature_name = gbm.feature_name()
        feature_importance = pd.DataFrame({'feature_name':feature_name,'importance':importance} ).sort_values(by='importance',ascending=False)
#         feature_importance.to_csv('feature_importance_{}.csv'.format(idx+1),index=None,encoding='utf-8-sig')
        print(feature_importance.head(10))
        test_pre = gbm.predict(test_user, num_iteration=gbm.best_iteration)
        test_pre = MinMaxScaler().fit_transform(test_pre.reshape(-1, 1))
        test_pre = test_pre.reshape(-1, )
        test_p.append(test_pre)
    threshold = threshold/len(test_p)
    print(threshold)
    print("五折平均分数: ", scores/len(test_p))
    test_p = np.array(test_p)
    test_p = test_p.mean(axis=0)
    sub["prob"] = test_p
    sub["label"] = sub["prob"] > round(np.percentile(sub["prob"], threshold), 4)
    sub[["phone_no_m", "label"]].to_csv('submission.csv', index=False,encoding='utf-8')
    return sub

In [61]:
# %%time

sub = train_model()

label    1962
dtype: int64 (6106, 1)    label
0      0
1      0
2      0
3      0
4      0
Training until validation scores don't improve for 200 rounds


/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:842: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))


[100]	training's auc: 0.999395	valid_1's auc: 0.957624
[200]	training's auc: 1	valid_1's auc: 0.958416
[300]	training's auc: 1	valid_1's auc: 0.958548
[400]	training's auc: 1	valid_1's auc: 0.958327
Early stopping, best iteration is:
[271]	training's auc: 1	valid_1's auc: 0.958885
score:  0.8907103825136612 72.24000000000001
             feature_name  importance
3             arpu_202004         447
43          calltype_rate         381
1             county_name         342
20    phone2opposite_mean         340
41               sms_rate         304
37     voc_day_mode_count         301
17          voc_calltype1         236
24  phone2oppo_sum_median         226
16                  imeis         223
36           voc_day_mode         222
Training until validation scores don't improve for 200 rounds


/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:842: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))


[100]	training's auc: 0.999397	valid_1's auc: 0.961593
[200]	training's auc: 1	valid_1's auc: 0.961279
Early stopping, best iteration is:
[28]	training's auc: 0.978461	valid_1's auc: 0.968863
score:  0.8891820580474934 70.08
           feature_name  importance
20  phone2opposite_mean          66
16                imeis          63
43        calltype_rate          59
3           arpu_202004          58
35     voc_hour_nunique          54
11            month_ids          45
37   voc_day_mode_count          44
41             sms_rate          36
38      voc_day_nunique          36
40          sms_nunique          23
Training until validation scores don't improve for 200 rounds


...

[100]	training's auc: 0.999386	valid_1's auc: 0.95438
[200]	training's auc: 1	valid_1's auc: 0.954765
[300]	training's auc: 1	valid_1's auc: 0.956008
[400]	training's auc: 1	valid_1's auc: 0.955823
[500]	training's auc: 1	valid_1's auc: 0.955799
Early stopping, best iteration is:
[306]	training's auc: 1	valid_1's auc: 0.95638
score:  0.88 70.72
             feature_name  importance
3             arpu_202004         541
43          calltype_rate         405
1             county_name         389
20    phone2opposite_mean         362
41               sms_rate         338
37     voc_day_mode_count         316
17          voc_calltype1         301
16                  imeis         270
24  phone2oppo_sum_median         254
35       voc_hour_nunique         243
Training until validation scores don't improve for 200 rounds


/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:842: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))


[100]	training's auc: 0.999344	valid_1's auc: 0.963812
[200]	training's auc: 1	valid_1's auc: 0.961252
Early stopping, best iteration is:
[70]	training's auc: 0.99548	valid_1's auc: 0.96492
score:  0.8956406869220608 70.16
           feature_name  importance
3           arpu_202004         143
16                imeis         117
43        calltype_rate         117
37   voc_day_mode_count         114
20  phone2opposite_mean         114
41             sms_rate          91
35     voc_hour_nunique          91
38      voc_day_nunique          80
11            month_ids          79
48       day_mode_count          66
Training until validation scores don't improve for 200 rounds


/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:842: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))


[100]	training's auc: 0.999359	valid_1's auc: 0.958993
[200]	training's auc: 1	valid_1's auc: 0.957564
Early stopping, best iteration is:
[89]	training's auc: 0.998597	valid_1's auc: 0.959346
score:  0.8994565217391306 71.92
           feature_name  importance
3           arpu_202004         179
20  phone2opposite_mean         163
16                imeis         133
43        calltype_rate         133
41             sms_rate         120
37   voc_day_mode_count         114
35     voc_hour_nunique         101
1           county_name          92
38      voc_day_nunique          85
40          sms_nunique          72
71.024
五折平均分数:  0.8909979298444691


In [55]:
sub.label.value_counts()

False    1452
True      593
Name: label, dtype: int64

## train_model_no_score_vail

In [35]:
def train_model_no_score_vail():
    test_app_feat=pd.read_csv(user_data+'test_app_feat.csv')
    test_voc_feat=pd.read_csv(user_data+'test_voc_feat.csv')
    test_sms_feat=pd.read_csv(user_data + "test_sms_feat.csv")
    test_user=pd.read_csv(path+'test/test_user.csv')
    test_user = test_user.merge(test_app_feat, on="phone_no_m", how="left")
    test_user = test_user.merge(test_voc_feat, on="phone_no_m", how="left")
    test_user = test_user.merge(test_sms_feat, on="phone_no_m", how="left")
    test_user["city_name"] = LabelEncoder().fit_transform(test_user["city_name"].astype(np.str))
    test_user["county_name"] = LabelEncoder().fit_transform(test_user["county_name"].astype(np.str))
    
    train_app_feat = pd.read_csv(user_data + "train_app_feat.csv")
    train_voc_feat = pd.read_csv(user_data + "train_voc_feat.csv")
    train_sms_feat=pd.read_csv(user_data + "train_sms_feat.csv")
    train_user=pd.read_csv(path+'train/train_user.csv')
    drop_r = ["arpu_201908","arpu_201909","arpu_201910","arpu_201911","arpu_201912","arpu_202001","arpu_202002"]
    train_user.drop(drop_r, axis=1,inplace=True)
    train_user.rename(columns={"arpu_202003":"arpu_202004"},inplace=True)
    train_user = train_user.merge(train_app_feat, on="phone_no_m", how="left")
    train_user = train_user.merge(train_voc_feat, on="phone_no_m", how="left")
    train_user = train_user.merge(train_sms_feat, on="phone_no_m", how="left")
    train_user["city_name"] = LabelEncoder().fit_transform(train_user["city_name"].astype(np.str))
    train_user["county_name"] = LabelEncoder().fit_transform(train_user["county_name"].astype(np.str))

    sub = test_user[["phone_no_m"]].copy()
    
    params = {
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'max_bin': 50,
    'max_depth': 16,
    "learning_rate":0.05,
    "colsample_bytree":0.8,#每次迭代中随机选择特征的比例
    "bagging_fraction":0.8, #每次迭代时用的数据比例
    'min_child_samples': 25,
    'n_jobs': -1,
    'silent': True,    #信息输出设置成1则没有信息输出
    'seed':1024,
    }
    train_label = train_user[["label"]].copy()
    print(np.sum(train_label), train_label.shape, train_label.head())
    
    test_user.drop(["phone_no_m"], axis=1,inplace=True)
    train_user.drop(["phone_no_m", "label"], axis=1,inplace=True)
    kf = StratifiedKFold(n_splits = 5, shuffle=True, random_state=2020)
    test_p = []
    threshold = 0
    scores = 0
    for i, (train_index, vaild_index) in enumerate(kf.split(train_user, train_label["label"])):
#         print('第{}次训练...'.format(i+1))
        train_x = train_user.iloc[train_index]
        train_y = train_label.iloc[train_index]
        valid_x = train_user.iloc[vaild_index]
        valid_y = train_label.iloc[vaild_index]
        
        cat = ["city_name", "county_name"]
        lgb_train = lgb.Dataset(train_x, train_y, categorical_feature=cat, silent=True)
        lgb_eval = lgb.Dataset(valid_x, valid_y, reference=lgb_train, categorical_feature=cat, silent=True)
        
        gbm = lgb.train(params, lgb_train, num_boost_round=1000, valid_sets=[lgb_train,lgb_eval], verbose_eval=100, early_stopping_rounds=200)
    
        vaild_preds = gbm.predict(valid_x, num_iteration=gbm.best_iteration)
        vaild_preds = pd.Series(vaild_preds).map(round).tolist()
        bestf1 = f1_score(vaild_preds, valid_y)
        bp = 0.5
#         bp, bestf1= score_vail(vaild_preds, valid_y)
        print("score: ", bestf1 , bp)
        scores += bestf1
        threshold += bp
#         sauc = myFevalAuc(vaild_preds, valid_y)
#         importance = gbm.feature_importance()
#         feature_name = gbm.feature_name()
#         feature_importance = pd.DataFrame({'feature_name':feature_name,'importance':importance} ).sort_values(by='importance',ascending=False)
#         feature_importance.to_csv('feature_importance_{}.csv'.format(idx+1),index=None,encoding='utf-8-sig')
        test_pre = gbm.predict(test_user, num_iteration=gbm.best_iteration)
        test_pre = MinMaxScaler().fit_transform(test_pre.reshape(-1, 1))
        test_pre = test_pre.reshape(-1, )
        test_p.append(test_pre)
    threshold = threshold/len(test_p)
    print(threshold)
    print("五折平均分数: ", scores/len(test_p))
    test_p = np.array(test_p)
    test_p = test_p.mean(axis=0)
    sub["prob"] = test_p
    sub["label"] = sub["prob"] > round(np.percentile(sub["prob"], threshold), 4)
    sub[["phone_no_m", "label"]].to_csv('submission.csv', index=False,encoding='utf-8')
    return sub

In [36]:
train_model_no_score_vail()

label    1962
dtype: int64 (6106, 1)    label
0      0
1      0
2      0
3      0
4      0
Training until validation scores don't improve for 200 rounds


/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:842: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))


[100]	training's auc: 0.999395	valid_1's auc: 0.957624
[200]	training's auc: 1	valid_1's auc: 0.958416
[300]	training's auc: 1	valid_1's auc: 0.958548
[400]	training's auc: 1	valid_1's auc: 0.958327
Early stopping, best iteration is:
[271]	training's auc: 1	valid_1's auc: 0.958885
score:  0.8894952251023193 0.5
Training until validation scores don't improve for 200 rounds


/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:842: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))


[100]	training's auc: 0.999397	valid_1's auc: 0.961593
[200]	training's auc: 1	valid_1's auc: 0.961279
Early stopping, best iteration is:
[28]	training's auc: 0.978461	valid_1's auc: 0.968863
score:  0.8680851063829786 0.5
Training until validation scores don't improve for 200 rounds


...

/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:842: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))


[100]	training's auc: 0.999344	valid_1's auc: 0.963812
[200]	training's auc: 1	valid_1's auc: 0.961252
Early stopping, best iteration is:
[70]	training's auc: 0.99548	valid_1's auc: 0.96492
score:  0.8855989232839838 0.5


/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:842: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))


Training until validation scores don't improve for 200 rounds
[100]	training's auc: 0.999359	valid_1's auc: 0.958993
[200]	training's auc: 1	valid_1's auc: 0.957564
Early stopping, best iteration is:
[89]	training's auc: 0.998597	valid_1's auc: 0.959346
score:  0.8958904109589041 0.5
0.5
五折平均分数:  0.8829490682807724


,phone_no_m,prob,label
0,22d522340df77e2252c1a4d92b4bcb00d515e36f3ec6bf94e017b4ffd67e26520af18637ad91bde69abd831cc36cdc0c...,0.600820,True
1,5220d4b8429bdba3971a7b46a088c6b8fa6710f40607598ac1219daab6071510b3f66f78637420574dbb2d5488d44de9...,0.998430,True
2,7d19dd2b50ced56f03d23bf928cf34dc570a48525571a868ffdcdc53a865b7a41c386a6f1d6567dd4a710530b933308c...,0.053393,True
3,134a4a591185c9d3788021896dcfc235e9e0a6a1e3f8a4fd3a9f9d9bf4a033e791b7199db980b0678020bdf689d71930...,0.535508,True
4,2356dcd6759d50455ddaeed03c838843558e9182d5962f8a4c81a9a178063c49ab988750f795bb6c5462aee5dace2ad9...,0.171873,True
5,4ea585532dd16a41b42f814c5ed2d18e93d07b2d76d770037fb477d30b46f46a142bcbb71bf81235879c15ea50520224...,0.726735,True
6,bd954fbc4a32ee202578d24c1af2df9e1db740912cead08e62656b4d7f90bbf839007576f523b5dc8be16de3e59d0743...,0.005907,True
7,ba64696f2808101fe7e7f37bb50117a8a72535bd516c9276a719cdcacee5d1813bc598f3d86d19dc0b4a0084c7b8c22c...,0.899975,True
8,6562edeaa63dcd67825cdb18e0d4f3e9eaa07fce0edcdfd456c901c5964292cd79aafb661cc3b58b18cd5d3ed0a2477e...,0.994285,True
9,32dee85edbe2423cf1f265133ae00fb833622bc59b7f05ef45cc06ee8c34bceaa904952c1fd0f389353a4ae13a88e1fc...,0.014516,True
